# KRAB-ZFP ChIP-seq, RNA-seq and evolutionary conservation analysis using Python scripts

## 1. Introduction

Krüppel-associated box zinc finger proteins (KRAB-ZFPs) constitute the largest transcription factor family in mammals. While most transcription factors bind relatively short DNA sequences present in the genome in thousands of copies, KRAB-ZFPs use a modular DNA binding domain, consisting of up to several dozen individual C2H2-type zinc fingers, allowing highly specific binding to long stretches of DNA that are unlikely to appear by chance. This mode of DNA binding, together with their KRAB repressor domain, enables KRAB-ZFPs to accurately and potently silence target genes. Interestingly, the KRAB-ZFP family rapidly amplified and diversified in mammals by segmental gene duplications, mutations, and zinc finger rearrangements. Recent evidence suggests that KRAB-ZFP diversification reflects an ongoing arms race between the host and transposable elements (TEs) that continuously invade and amplify within their host genomes. While this hypothesis might explain some aspects of KRAB-ZFP function and evolution, many questions remain open.

To address these questions, we initiated a medium-scale screen to identify the genome-wide binding sites of more than 50 previously uncharacterized mouse KRAB-ZFPs. Additionally, we gereated five KRAB-ZFP cluster KO ES cell lines, altogether genetically deleting nearly 100 KRAB-ZFPs. These cell lines were thoroughly analyzed by ChIP-seq, RNA-seq and other approaches. Additionally, we generated two cluster KO mouse lines in which we peformed RNA-seq in various tissues.

## 2. Aim

To generate simple, fast and reporoducible workflows, I wrote several Python scripts that process data that were generated other tools such as Bowtie, MACS14, DESeq and various Bedtools functions. The final goal is to to combine all scripts in one single program that processes raw ChIP-seq and RNA-seq files to generate publication-ready figures. However, some of these datasets are very big and the incorporation of other tools (which often take a long time to run) into a single Python script is challenging. Therefore, for now, I will use pre-written computational tools such as Bowtie and MACS14 in commandline, using awk commands and custom unix shell executables files in order to process a large amount of data efficiently and in relatively short time (e.g. mapping a typical ChIP-seq file using Bowtie, including sam to bam conversion and sorting of the final output file, takes less than 10 minutes when using 24 processors on our local server.) The aim for each of the developed Python scripts is explained in more detail below, including a workflow scheme that lists all other tools that are required to generate the input files for the Python scripts.


# 3. Python scripts for data analysis

## 3.1 MACS14 processing

I used Bowtie (--best mode) to map ChIP-seq fastq files to the mouse genome (mm9) and MACS14 (-p 1E-10) to call regions with significant enrichment of ChIP sample vs Input control (peaks). MACS14 reports the results as excel files that contain all ChIP-seq peak information. The purpose of this code is to filter out peaks that are less than 20-fold enriched over Input and to add an additional column that states the KRAB-ZFP from for which the ChIP-seq peaks have been determined. The results are saved as one concatenated bed file for further processing.                                                            

In [1]:
# import modules (note: since all  scripts are working independantly, modules are imported for each module even if they already have been loaded for a previous script)
import pandas as pd
import glob

In [56]:
files = glob.glob('peaks/*.xls')    # create list with file names containing all .xls files with the 'peak' folder in the working directory 
print('Found .xls files:',files)    

for file in files:    # iterate through list with file names
    
    df = pd.read_csv(file, skiprows=23, sep='\t')       # load file as panda dataframe
    new_name = file.split('\\')[1]                      # split file name to remove path from name
    df['Zfp_name'] = new_name.split('_')[0]             # extract ZFP name from file name and make new column with it
    df_sorted = df.sort_values('fold_enrichment', ascending=False)       # sort dataframe according to fold_enrichment column
    df_sorted_filtered = df_sorted[df_sorted['fold_enrichment'] >= 20]   # filter dataframe to keep only peaks with >= 20-fold enrichment
    new_file_name = file[:-4] + '.bed'                                   # create new file name to save dataframe (as bed file)
    df_sorted_filtered.to_csv(new_file_name, sep='\t', header=False, index=False)    # save dataframe as csv file in bed format

# Concatenate peak files into one file

results = pd.DataFrame([])
files = glob.glob('peaks/*peaks.bed')

for file in files:
    namedf = pd.read_csv(file, sep='\t', header=None)
    results = results.append(namedf)

no_of_ZFPs = str(len(files))                                        # convert number of processed files from int to str
output_name = 'peaks/' + no_of_ZFPs + '_KRABs.bed'                  # create output file name (with path to peak folder)
results.to_csv(output_name, sep='\t', header=False, index=False)    # save concatenated dataframe as bed file               

Found .xls files:	 ['peaks\\Gm13150_Input_peaks.xls', 'peaks\\Zfp273_Input_peaks.xls', 'peaks\\Zfp456_Input_peaks.xls', 'peaks\\Zfp457_Input_peaks.xls', 'peaks\\Zfp58_Input_peaks.xls', 'peaks\\Zfp712_Input_peaks.xls', 'peaks\\Zfp738_Input_peaks.xls', 'peaks\\Zfp92_Input_peaks.xls']


## 3.2 Find targeted TEs

To find out whether these KRAB-ZFPs preferably target transposable elements, I use the 'Join the intervals of two datasets side-by-side' script on Galaxy. The tools finds overlapping regions in two interval or bed files and returns a table with regions of the first dataset (in this case the ChIP-seq peak file) and any overlapping regions of the second dataset (a bed file with all transposon coordinated from the UCSC Table Browser) in one line. The following scripts reports transposon families if they (1) overlap with at least three peaks of a given KRAB-ZFP and (2) the peaks overlapping with the transposon family make up at least two percent of all peaks. 

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [65]:
df = pd.read_csv('input_files/49_KRAB-ZFP_peaks_200bp_TE.interval', sep='\t', header=None, names=['Chr', 'Start', 'End', 'Length', 'Summit', 'Tags', 'Enrichment', 'FDR', 'ZFP', 'TE'])
unique_ZFPs = set(df.ZFP.tolist())    # crate list with unique ZFP names

for zfp in unique_ZFPs:               # iterate through ist with unique ZFP names
    
    df_zfp = df[df['ZFP'] == zfp]                       # filter entries for matching ZFP name (create new dataframe for each ZFP)
    TE_count = df_zfp.TE.value_counts()                 # count how many peaks overlap with eah TE      
    ZFP_count = df_zfp.ZFP.value_counts()               # count ZFP peaks
    Zfp_number = ZFP_count[0]
    percent_TE = TE_count / Zfp_number * 100            # calculate percentage of peaks that overlap with TE
    merged = pd.concat([TE_count, percent_TE], axis=1)
    merged.columns = ['TE_count', 'TE_percentage']
    merged = merged[merged['TE_count'] >= 3]            # filter TEs with at least 3 overlapping peaks
    merged = merged[merged['TE_percentage'] >= 2]       # filter TEs that overlap with at least 2 percent of peaks
    print(zfp)
    print(merged, '\n')

Gm13251
    TE_count  TE_percentage
.       3990      64.096386
B3       147       2.361446 

Gm13154
        TE_count  TE_percentage
.           1304      70.869565
RLTR9E       130       7.065217 

Zfp595
   TE_count  TE_percentage
.        16      76.190476 

Gm30910
             TE_count  TE_percentage
MMERGLN-int        29      38.157895
MuRRS4-int         19      25.000000
.                  10      13.157895
MURVY-int           7       9.210526
RLTR6-int           6       7.894737 

Zfp738
           TE_count  TE_percentage
.               138      78.409091
MERVL-int         7       3.977273
MMETn-int         6       3.409091
RMER19B           5       2.840909 

Gm14399
           TE_count  TE_percentage
.               141      43.788820
IAPEz-int        86      26.708075
L1Md_F2           7       2.173913
L1Md_A            7       2.173913 

Gm13152
          TE_count  TE_percentage
.             4562      75.906822
RLTR13B1       316       5.257903
RLTR13A        136       2

## 3.3 Determine KAP1 and H3K9me3 enrichment at TE peaks

This script uses a bed file that contains all KRAB-ZFP peaks with overlapping TEs (as used above). Additionally, this file was used to count all overlapping reads from KAP1 and H3K9me3 ChIP-seq data (generated by Bedtools multicov). The second input file (fisher_matrix_TE_only.txt) was generated using Fisher Bed, which calculates the significance of overlaps between two bed files.

In [27]:
# set read numbers to calculate RPM (determined by analyzing bam files using bamtools: Flagstat)
KAP1_ChIP = 61808121      
KAP1_Input = 115085987
H3K9me3_ChIP = 43340454
H3K9me3_Input = 46710977

# load data with KAP1 and H3K9me3 enrichment as dataframe
df = pd.read_csv('input_files/49_KRAB-ZFP_peaks_200bp_TE_KAP1_H3K9me3.interval', sep='\t', header=None, names=['Chr', 'Start', 'End', 'Enrichment', 'ZFP', 'TE', 'KAP1_ChIP', 'KAP1_Input', 'H3K9me3_ChIP', 'H3K9me3_Input'])

# generate set with unique ZFP names
unique_ZFPs = set(df.ZFP.tolist())     

# Load data with fisher bed values as dataframe
matrix = pd.read_csv('input_files/fisher_matrix_TE_only.txt', sep='\t')

# calculate RPM values
df['KAP1_ChIP'] = df['KAP1_ChIP'] / KAP1_ChIP * 1000000
df['KAP1_Input'] = df['KAP1_Input'] / KAP1_Input * 1000000
df['H3K9me3_ChIP'] = df['H3K9me3_ChIP'] / H3K9me3_ChIP * 1000000
df['H3K9me3_Input'] = df['H3K9me3_Input'] / H3K9me3_Input * 1000000

KAP1_enrichment_list = []
H3K9me3_enrichment_list = []

# Iterate through unique ZFP set
for zfp in unique_ZFPs:
    
    df_zfp = df[df['ZFP'] == zfp]                    # Filter read count table for ZFP name
    matrix_zfp_sign = matrix[matrix[zfp] < 1e-5]    # Filter fisher bed table for significant overlapping TEs     
    TE_list = matrix_zfp_sign.Repeat.tolist()        # Make list of TEs that are significantly bound by ZFP
    if TE_list == []:
        KAP1_enrichment_list.append('n/a')
        H3K9me3_enrichment_list.append('n/a')
        continue
    
    KAP1_ChIP_TE = []       
    KAP1_Input_TE = []
    H3K9me3_ChIP_TE = []
    H3K9me3_Input_TE = []
    
    # Iterate through list of TEs that are significantly bound by ZFPs and add KAP1 and H3K9me3 read count
    for TE in TE_list:
        
        df_zfp_TE = df_zfp[df_zfp['TE'] == TE]                  
        
        KAP1_ChIP_TE += df_zfp_TE['KAP1_ChIP'].tolist()
        KAP1_Input_TE += df_zfp_TE['KAP1_Input'].tolist()
        H3K9me3_ChIP_TE += df_zfp_TE['H3K9me3_ChIP'].tolist()
        H3K9me3_Input_TE += df_zfp_TE['H3K9me3_Input'].tolist()
    
    # Add sum KAP1 and H3K9me3 reads to new lists
    KAP1_enrichment_list.append(sum(KAP1_ChIP_TE)/sum(KAP1_Input_TE))
    H3K9me3_enrichment_list.append(sum(H3K9me3_ChIP_TE)/sum(H3K9me3_Input_TE))
      
Table = pd.DataFrame(KAP1_enrichment_list,H3K9me3_enrichment_list)

combined = zip(KAP1_enrichment_list, H3K9me3_enrichment_list)
dic = dict(zip(unique_ZFPs, combined))
df = pd.DataFrame(dic, index=['KAP1_enrichment', 'H3K9me3_enrichment'])
print(df)
df.to_csv('KAP1_H3K9me3_enrichment_TEs', sep='\t')

                    AW146154    Gm13051   Gm13139    Gm13150   Gm13152  \
KAP1_enrichment     0.916066  19.843509  9.918269  27.471873  4.799651   
H3K9me3_enrichment  0.731429   8.628025  5.568266  14.173394  3.097382   

                     Gm13154   Gm13157 Gm13225   Gm13251   Gm14295    ...     \
KAP1_enrichment     6.608769  6.184005     n/a  6.230552  3.430649    ...      
H3K9me3_enrichment  3.082589  7.484725     n/a  3.310657  1.472444    ...      

                      Zfp810   Zfp874a   Zfp874b    Zfp882  Zfp92    Zfp931  \
KAP1_enrichment     0.688784  2.241282  3.103314  2.768353    n/a  7.053676   
H3K9me3_enrichment  0.481556  2.634840  4.372661  1.142113    n/a  3.601324   

                      Zfp935    Zfp938  Zfp953    Zfp961  
KAP1_enrichment     3.578508  3.583449     n/a  7.694648  
H3K9me3_enrichment  5.963652  1.244091     n/a  6.427952  

[2 rows x 49 columns]


## 3.4 Create Scatter plots with DESeq output files

DESeq determines genes or other elements whose expression is significantly changed when comparing two conditions. The ouput is reported in a table. The purpose of this script is to generate scatter plots from multiple DESeq tables that have been pre-processed. Included in the script is a loop that iterated through the entries and assigns colors for all significantly up- and down-regulted genes (in this case retrotransposons).

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [32]:
files = glob.glob('deseq_files/*.interval')    # create list with filenames (pre-processed output files from DESeq) of all .interval files in the deseq_files folder in the directory

index = 0
for file in files:                            # iterate through list with file names
    file_name = files[index]
    colnames = ['Chr', 'Start', 'End', 'Strand', 'Transposon', 'Mean', 'log2fold', 'x1', 'x2', 'pvalue', 'qvalue']
    df = pd.read_csv(file_name, sep = '\t', names=colnames)

    color = []                         # assign colours (red/blue) to up- and downregulated TEs, save as list in 'color'
    for row in df.itertuples():
        if row[7] > 0 and row[11] < 0.05:    # transposons that are significantly upregulated will be shown in red
            color.append('red')
        elif row[7] < 0 and row[11] < 0.05:  # transposons that are significantly downregulated will be shown in blue
            color.append('blue')
        else: 
            color.append(0.5)                # not significantly changed transposons will be shown in grey

    df.plot(x='Mean', y='log2fold', kind='scatter', s=1, c=color)   # crate scatter plot and save as file
    plt.xscale('log') 
    plt.xlabel('Mean TE expression')
    plt.ylabel('Log2-fold change')
    plt.title(file_name[12:-9])                  # plot title is input file name without extension
    plt.axhline(color='black')
    plt.savefig(file_name[:-9])
    plt.close()
    index += 1

## 3.5 Extract amino acids responsible for DNA specificity from C2H2 zinc finger domains

C2H2 zinc finger domains form a loop stabilized by a zentral zinc ion that contacts both cystidine and histidine amino acids. The amino acids that are mainly responsible for DNA binding specificity are located at defined positions within the finger. 

### C--C-----X--X--XH---H

C: Cystidine; 
H: Cystidine; 
-: Variable amino acid; 
X: Contact amino acids

For evolutionary conservation analysis in KRAB-ZFPs, it is often better to only consider these contact amino acids since they indicate whether the DNA binding preference is conserved. This script uses a multiple fasta file with protein sequences (stop codons indicated by asterisks), identifies C2H2 zinc fingers and returns the contact amino acids for each zinf finger in a fasta-like output. This output file can be loaded into other programs to make alignments or perform BLAST screens.

In [33]:
def find_aa(x):  
    
    """" this function screens a protein sequence (as string) for occurences of C2H2 zinc finger
    motifs and returns a string with the three main contact amino  acids of each finger, separated by a dash """
    
    seq = str.upper(x)
    fingerprint = '' 
    while len(seq) > 21:                                                   # loop as long the end of the sequence is > 21 aa
        if seq[0] + seq[3] + seq[16] + seq[20] == 'CCHH':
            fingerprint = fingerprint + seq[9] + seq[12] + seq[15] + '-'
            seq = seq[20:]                                                 # Skip remaining C2H2 motif for further screen
        seq = seq[1:]                                                      # Remove fist amino acid from sequence
    return fingerprint[:-1]

with open('zfp_files/KRAB-ZFPs.fa') as file:                 # Open fasta file with protein sequences
    
    zfp_seq = ''                                   # Empty string for protein sequence
    zfp_names = []                                 # Empty list for protein names
    
    for line in file:                              # iterate through lines of fasta file
        
        if line[0] != '>':                         # Add protein sequence to string if line belongs to sequence  
            zfp_seq += line
            
        else:                                      # Store protein name in list if line starts with '>'
            zfp_names.append(line)

Zfp_seq_flat = zfp_seq.replace('\n', '')           # remove newlines from sequence
zfp_seq_list = Zfp_seq_flat.split('*')             # split sequence to individual protein sequences

for i in range(len(zfp_seq_list[:-1])):            # Iterate through protein sequences and apply find_aa function for each seq
    print(zfp_names[i], find_aa(zfp_seq_list[i]))

>ZFP809
 WDS-RHA-YDR-RKI-HQA-WQA
>Zfp599
 SKL-HYQ-RHL-CQL-YHR-CDV-QKL-CHL-YQV
>Zfp810
 CHV-SQV-RDV-CQV-CQV-CDR-RDV-CQV-CER-TDR
>Zfp961
 CSK-YCR-NAE-TSM-RSL-HHA-SCR-YSN-YSN-CYN-YSR
>Zfp882
 SSE-SST-YGR-DSC-DSC-RSC-HAY-LAK-FSC-YSC-YSR-DSC-TSY
>Zfp709
 FNR-CNE-FNR-YND-YST-YST-TSY-STI-VSC-RSC-SNK-GSR-SGR-NAN-SKI-GSR-TDR-RSI
>Zfp617
 YSN-RNA-STA-STA-TTA-STA-QYI-RHI-QYI-RHI-HYI-RSI-LSR-CSK
>Zfp600
 DTK-DTK-DSV-QHI-DTK-ETK-DSV-KHI-QPI-DSV-QHI-DSV-DSV-DYV-HSV-RHI-DSR
>2610305D13Rik
 QLN-HSI-HSI
>Gm13051
 NNM-KNI-QYI-ESI-QHI-QNI-ESI-ESI-ESI-VSI-KHI-EII-RHI-QHI
>Gm13139
 HHI-QHI-ENI-QHI-EHI-EHI-KSI-FSI-ECI-FSI-EKI
>Gm13212
 HHI-HSM-ESI-HHS-KSI-QDI-HHS-RHI-QDI-HHS-RHI-RHI-RHI-RHI-QDI-HHS-RHI-QNI-ESI-RHI
>Gm13242
 DTK-DSV-QHI-DSV-DSV-DSV-HSV-EGR-DSV-DSV-DYV-HSV-EGR
>Rex2_
 DTK-DTK-DSV-QHI-DTK-ETK-DSV-KHI-QPI-DSV-QHI-HSV-EGR-RHI-QHI
>Gm13154
 HNA-HRY-HNA-HNA-RHI-GSI-GSI-GSI-QHI-HHN
>Gm13157
 HHI-HSM-QHV-HHS-KSI-HNI-HHS-RHI-QNI-ESI-RHI
>LOC102638055
 ESI-ESI-ESI-VSI-KNI-RHQ-DSV-QHI-DSV-DSV-DYV-HSV
>

# 4. Discussion and outlook

Python is a powerful programming language to perform computational analysis of data generated by next generation sequencing technologies. The short scripts described above can significantly shorten the time it takes to process data files generated by programs previously developed for ChIP-seq and RNA-seq. Importantly, the scripts can be re-used when more next generation sequencing data is constantly produced over the course of a project. In my case, I started with only a handful of ChIP-seq data but these datasets kept growing for more than two years. The developed scripts will help me to analyse new data in very little time in a reproducible manner. I also have previously spent a lot of time to generate RNA-seq scatter plots using excel which is not only time consuming and often leads to a crash of excel because of the large amounts of data, but is also not very useful when trying to generate figures from several datasets. The very short and simple python script to create scatter plots from DESeq2 output files is therefore a very helpful tool for me. 
Finally, the script that extracts contact amino  acids from C2H2 zinc finger proteins, although already proven to be useful for my specific purposes, could be extended in several ways. One interesting possibility would be to use an improved version to screen entire genomes for the presence of these C2H2 proteins and provide various information about the matches. Such a script could load batched of sequenced genomes (ideally without the need to save the data locally), translate the DNA sequence to protein in all 6 possible open reading frames, and screen the protein sequences for C2H2 fingers. Furthermore, such a script could easily be modified to look for all kind of protein or DNA motifs in a large number of genomes.
The next logical step is to combine all the individual scripts in one programm that is able to receive files from other programs, potentiall not written in Python, process the files, return them to a new program and so on. That way, one could generate a pipeline that creates final figures from the raw sequencing files in one run. Nevertheless, since some of the processing steps are time consuming (e.g. mapping), it might make sense to split the script in two or more pieces.